Data extracted from the Harvard Business School article 'HR Analytics at ScaleneWorks: Behavioral Modelling to Predict Renege'.
The goal of this project is to build a model to predict if the candidate will join the company after a job offer using both a KNN model and a Logistic Regression model and discuss the difference in results.

In [156]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

data = pd.read_excel('../Scalene_Works.xlsx', sheet_name=1)

In [153]:
data.head()

,DOJ Extended,Duration to accept offer,Notice period,Percent hike expected in CTC,Percent hike offered in CTC,Percent difference CTC,Joining Bonus,Candidate relocate actual,Rex in Yrs,Age,Status,Gender_Male,Candidate_Source_Agency,Candidate_Source_Direct,Offered_band_E0,Offered_band_E1,Offered_band_E2,Offered_band_E3,Offered_band_E4,Offered_band_E5
0,1,14.0,30,-20.79,13.16,42.86,0,0,7,34,1,0,1,0,0,0,1,0,0,0
1,0,18.0,30,50.00,320.00,180.00,0,0,8,34,1,1,0,0,0,0,1,0,0,0
2,0,3.0,45,42.84,42.84,0.00,0,0,4,27,1,1,1,0,0,0,1,0,0,0
3,0,26.0,30,42.84,42.84,0.00,0,0,4,34,1,1,0,0,0,0,1,0,0,0
4,1,1.0,120,42.59,42.59,0.00,0,1,6,34,1,1,0,0,0,0,1,0,0,0


In [120]:
data.columns

Index(['Candidate Ref', 'DOJ Extended', 'Duration to accept offer',
       'Notice period', 'Offered band', 'Pecent hike expected in CTC',
       'Percent hike offered in CTC', 'Percent difference CTC',
       'Joining Bonus', 'Candidate relocate actual', 'Gender',
       'Candidate Source', 'Rex in Yrs', 'LOB', 'Location', 'Age', 'Status'],
      dtype='object')

DATA CLEANING / PREPROCESSING

In [121]:
# Transform 'DOJ Extended' into dummy
def YesNo_todummy(row):
    if row == 'Yes':
        return 1
    else:
        return 0
data['DOJ Extended'] = data['DOJ Extended'].apply(lambda row: YesNo_todummy(row))
# Transform 'Joining Bonus' into dummy
data['Joining Bonus'] = data['Joining Bonus'].apply(lambda row: YesNo_todummy(row))
# Transform 'Candidate relocate actual' into dummy
data['Candidate relocate actual'] = data['Candidate relocate actual'].apply(lambda row: YesNo_todummy(row))
# Transform 'Gender' into dummy 'Gender_Male' and drop original
def is_male(row):
    if row == 'Male':
        return 1
    else:
        return 0
data['Gender_Male'] = data['Gender'].apply(lambda row: is_male(row))
# Exclude original columns 'Candidate Ref', Gender', 'LOB', and 'Location'
data = data.drop(columns=['Candidate Ref', 'Gender', 'LOB', 'Location'])
# Rename 'Pecent hike expected in CTC' column
data = data.rename(columns={"Pecent hike expected in CTC": "Percent hike expected in CTC"})

data.head()

,DOJ Extended,Duration to accept offer,Notice period,Offered band,Percent hike expected in CTC,Percent hike offered in CTC,Percent difference CTC,Joining Bonus,Candidate relocate actual,Candidate Source,Rex in Yrs,Age,Status,Gender_Male
0,1,14.0,30,E2,-20.79,13.16,42.86,0,0,Agency,7,34,Joined,0
1,0,18.0,30,E2,50.00,320.00,180.00,0,0,Employee Referral,8,34,Joined,1
2,0,3.0,45,E2,42.84,42.84,0.00,0,0,Agency,4,27,Joined,1
3,0,26.0,30,E2,42.84,42.84,0.00,0,0,Employee Referral,4,34,Joined,1
4,1,1.0,120,E2,42.59,42.59,0.00,0,1,Employee Referral,6,34,Joined,1


In [122]:
data['Candidate Source'].value_counts()

Direct               7075
Agency               3170
Employee Referral    2088
Name: Candidate Source, dtype: int64

In [123]:
# Transform 'Candidate Source' into dummies (2 dummies)
Candidate_Source_dummies = pd.get_dummies(data['Candidate Source'])
data['Candidate_Source_Agency'] = Candidate_Source_dummies['Agency']
data['Candidate_Source_Direct'] = Candidate_Source_dummies['Direct']

# Exclude original 'Candidate Source' column
data = data.drop(columns=['Candidate Source'])

# Everything else has the value of 'Employee Referral' (if 0 in both dummies)

In [124]:
data['Offered band'].value_counts()

E1    6996
E2    3458
E0    1058
E3     690
E4     106
E5      23
E6       2
Name: Offered band, dtype: int64

In [125]:
# Transform 'Offered band' into dummies (6 dummies)
Offered_band_dummies = pd.get_dummies(data['Offered band'])
data['Offered_band_E0'] = Offered_band_dummies['E0']
data['Offered_band_E1'] = Offered_band_dummies['E1']
data['Offered_band_E2'] = Offered_band_dummies['E2']
data['Offered_band_E3'] = Offered_band_dummies['E3']
data['Offered_band_E4'] = Offered_band_dummies['E4']
data['Offered_band_E5'] = Offered_band_dummies['E5']

# Exclude original 'Offered band' column
data = data.drop(columns=['Offered band'])

# Everything else has the value of 'E6' (if 0 in all other 6 dummies)

In [126]:
data.head()

,DOJ Extended,Duration to accept offer,Notice period,Percent hike expected in CTC,Percent hike offered in CTC,Percent difference CTC,Joining Bonus,Candidate relocate actual,Rex in Yrs,Age,Status,Gender_Male,Candidate_Source_Agency,Candidate_Source_Direct,Offered_band_E0,Offered_band_E1,Offered_band_E2,Offered_band_E3,Offered_band_E4,Offered_band_E5
0,1,14.0,30,-20.79,13.16,42.86,0,0,7,34,Joined,0,1,0,0,0,1,0,0,0
1,0,18.0,30,50.00,320.00,180.00,0,0,8,34,Joined,1,0,0,0,0,1,0,0,0
2,0,3.0,45,42.84,42.84,0.00,0,0,4,27,Joined,1,1,0,0,0,1,0,0,0
3,0,26.0,30,42.84,42.84,0.00,0,0,4,34,Joined,1,0,0,0,0,1,0,0,0
4,1,1.0,120,42.59,42.59,0.00,0,1,6,34,Joined,1,0,0,0,0,1,0,0,0


In [127]:
data.describe()

,DOJ Extended,Duration to accept offer,Notice period,Percent hike expected in CTC,Percent hike offered in CTC,Percent difference CTC,Joining Bonus,Candidate relocate actual,Rex in Yrs,Age,Gender_Male,Candidate_Source_Agency,Candidate_Source_Direct,Offered_band_E0,Offered_band_E1,Offered_band_E2,Offered_band_E3,Offered_band_E4,Offered_band_E5
count,12333.000000,9614.000000,12333.000000,11586.000000,11737.000000,11482.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000,12333.000000
mean,0.411417,21.189619,37.943323,41.837663,37.772016,-2.078916,0.043217,0.132166,4.160788,30.232304,0.814644,0.257034,0.573664,0.085786,0.567259,0.280386,0.055947,0.008595,0.001865
std,0.492110,26.410351,24.526587,29.435844,35.593317,19.997159,0.203354,0.338685,2.810108,4.178853,0.388602,0.437016,0.494564,0.280059,0.495476,0.449206,0.229830,0.092313,0.043146
min,0.000000,-228.000000,0.000000,-68.830000,-60.530000,-68.950000,0.000000,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,30.000000,25.000000,20.000000,-9.090000,0.000000,0.000000,3.000000,27.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,9.000000,30.000000,38.890000,34.480000,0.000000,0.000000,0.000000,4.000000,30.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,32.000000,60.000000,52.170000,47.060000,0.000000,0.000000,0.000000,6.000000,34.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,224.000000,120.000000,359.770000,471.430000,300.000000,1.000000,1.000000,26.000000,64.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [128]:
# Check for variables which have null values
data.isnull().any()

# 'Duration to accept offer', 'Percent hike expected in CTC', 'Percent hike offered in CTC', and 'Percent difference CTC' 
# all have null values

DOJ Extended                    False
Duration to accept offer         True
Notice period                   False
Percent hike expected in CTC     True
Percent hike offered in CTC      True
Percent difference CTC           True
Joining Bonus                   False
Candidate relocate actual       False
Rex in Yrs                      False
Age                             False
Status                          False
Gender_Male                     False
Candidate_Source_Agency         False
Candidate_Source_Direct         False
Offered_band_E0                 False
Offered_band_E1                 False
Offered_band_E2                 False
Offered_band_E3                 False
Offered_band_E4                 False
Offered_band_E5                 False
dtype: bool

In [129]:
print("Data ratio with null values in 'Duration to accept offer': ", round(len(data[data['Duration to accept offer'].isna()]) / len(data),4))
print("Data ratio with null values in 'Percent hike expected in CTC': ", round(len(data[data['Percent hike expected in CTC'].isna()]) / len(data),4))
print("Data ratio with null values in 'Percent hike offered in CTC': ", round(len(data[data['Percent hike offered in CTC'].isna()]) / len(data),4))
print("Data ratio with null values in 'Percent difference CTC': ", round(len(data[data['Percent difference CTC'].isna()]) / len(data), 4))
print("Data ratio with null values in CTC-related columns: ", round(len(data[data['Percent hike expected in CTC'].isna() | data['Percent hike offered in CTC'].isna() | data['Percent difference CTC'].isna()]) / len(data), 4))

Data ratio with null values in 'Duration to accept offer':  0.2205
Data ratio with null values in 'Percent hike expected in CTC':  0.0606
Data ratio with null values in 'Percent hike offered in CTC':  0.0483
Data ratio with null values in 'Percent difference CTC':  0.069
Data ratio with null values in CTC-related columns:  0.0732


Since the data with null values for 'Duration to accept offer' are significant (around 22% of the data), deleting all such rows will result in the loss of valuable data. Therefore, null values for this column will be given a result equal to the mean of the column.

CTC-related columns with null values, on the other hand, only represent around 7% of the data. Therefore, the strategy taken in this anlysis will be to delete all rows with null values in any of these 3 columns.

In [130]:
# Convert 'Duration to accept offer' NAs to have the value of the column mean
data['Duration to accept offer'] = data['Duration to accept offer'].fillna(data['Duration to accept offer'].mean())

In [131]:
# Delete rows from CTC-related columns where the value is NA
data = data[data['Percent hike offered in CTC'].notna() & data['Percent difference CTC'].notna() & data['Percent hike expected in CTC'].notna()]

In [132]:
data.describe()

,DOJ Extended,Duration to accept offer,Notice period,Percent hike expected in CTC,Percent hike offered in CTC,Percent difference CTC,Joining Bonus,Candidate relocate actual,Rex in Yrs,Age,Gender_Male,Candidate_Source_Agency,Candidate_Source_Direct,Offered_band_E0,Offered_band_E1,Offered_band_E2,Offered_band_E3,Offered_band_E4,Offered_band_E5
count,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000,11430.000000
mean,0.425897,21.331381,37.695538,41.879379,37.865659,-2.088374,0.044707,0.121522,4.175941,30.309361,0.819598,0.269554,0.557830,0.079703,0.570866,0.283552,0.055556,0.008224,0.001925
std,0.494500,23.030428,23.974278,29.523014,35.575341,20.042107,0.206669,0.326748,2.773813,4.128334,0.384539,0.443747,0.496666,0.270844,0.494974,0.450742,0.229071,0.090316,0.043832
min,0.000000,-228.000000,0.000000,-68.830000,-60.530000,-68.950000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.000000,30.000000,25.000000,20.000000,-9.090000,0.000000,0.000000,3.000000,27.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,20.000000,30.000000,38.890000,34.260000,0.000000,0.000000,0.000000,4.000000,30.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,26.000000,60.000000,52.170000,47.200000,0.000000,0.000000,0.000000,6.000000,34.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,224.000000,120.000000,359.770000,471.430000,300.000000,1.000000,1.000000,26.000000,62.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [133]:
# Check if any NAs remain
data.isnull().any()

DOJ Extended                    False
Duration to accept offer        False
Notice period                   False
Percent hike expected in CTC    False
Percent hike offered in CTC     False
Percent difference CTC          False
Joining Bonus                   False
Candidate relocate actual       False
Rex in Yrs                      False
Age                             False
Status                          False
Gender_Male                     False
Candidate_Source_Agency         False
Candidate_Source_Direct         False
Offered_band_E0                 False
Offered_band_E1                 False
Offered_band_E2                 False
Offered_band_E3                 False
Offered_band_E4                 False
Offered_band_E5                 False
dtype: bool

In [138]:
# Transform 'Status' (our target variable) to a dummy
data['Status'] = data['Status'].apply(lambda row: 1 if row == 'Joined' else 0)

0        1
1        1
2        1
3        1
4        1
        ..
12328    1
12329    1
12330    1
12331    1
12332    1
Name: Status, Length: 11430, dtype: int64

In [154]:
# Scale data
scaler = MinMaxScaler()
col_names = ['Duration to accept offer', 'Notice period', 'Percent hike expected in CTC', 'Percent hike offered in CTC', 'Percent difference CTC', 'Rex in Yrs', 'Age']
data[col_names] = scaler.fit_transform(data[col_names])
data.head()

,DOJ Extended,Duration to accept offer,Notice period,Percent hike expected in CTC,Percent hike offered in CTC,Percent difference CTC,Joining Bonus,Candidate relocate actual,Rex in Yrs,Age,Status,Gender_Male,Candidate_Source_Agency,Candidate_Source_Direct,Offered_band_E0,Offered_band_E1,Offered_band_E2,Offered_band_E3,Offered_band_E4,Offered_band_E5
0,1,0.535398,0.250,0.112086,0.138525,0.303049,0,0,0.269231,0.333333,1,0,1,0,0,0,1,0,0,0
1,0,0.544248,0.250,0.277252,0.715336,0.674753,0,0,0.307692,0.333333,1,1,0,0,0,0,1,0,0,0
2,0,0.511062,0.375,0.260546,0.194319,0.186882,0,0,0.153846,0.166667,1,1,1,0,0,0,1,0,0,0
3,0,0.561947,0.250,0.260546,0.194319,0.186882,0,0,0.153846,0.333333,1,1,0,0,0,0,1,0,0,0
4,1,0.506637,1.000,0.259963,0.193849,0.186882,0,1,0.230769,0.333333,1,1,0,0,0,0,1,0,0,0


In [155]:
# Split data
X = data.drop(columns=['Status'])
y = data['Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [157]:
# Define k
print(math.sqrt(len(y_train))) # k = 89

89.44830909525345


In [176]:
# Define the model
knn = KNeighborsClassifier(n_neighbors = 89)

In [177]:
# Fit training data to the model
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=89)

In [178]:
# Predict test set results
y_pred = knn.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [179]:
# Evaluate the model (confusion matrix)
cm = confusion_matrix(y_test, y_pred)
print(cm)

# 'Not Joined' (value of 0) = 196 (predicted correctly), 706 (predicted incorrectly)
# 'Joined' (value of 1) = 103 (predicted incorrectly), 2424 (predicted correctly)
# https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

[[ 196  706]
 [ 103 2424]]


In [180]:
# Evaluate the model (f1 score)
print(f1_score(y_test, y_pred))

# Precision = 2424 / (2424 + 706) = ~.7744
# Recall = 2424 / (2424 + 103) = ~.9592
# F1 score = 2 * (Precision * Recall) / (Precision + Recall) = 2 * (.7744 * .9592) / (.7744 + .9592)
# This corresponds to the harmonic mean of precision and recall
# Therefore, the harmonic mean of precision and recall of our model is around 85.7%
# https://www.statology.org/f1-score-vs-accuracy/

0.8569913381651053


In [181]:
# Evaluate the model (accuracy score)
print(accuracy_score(y_test, y_pred))

# Accuracy score = (196 + 2424) / (196 + 706 + 103 + 2424)
# This corresponds to the total values predicted correctly divided by the total number of values predicted
# Therefore, our KNN model predicted around 76.4% of the values correctly

0.7640711577719451
